In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from IPython.display import clear_output
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import kit_function
from openpyxl import load_workbook

In [2]:
# Inicializa a instância do EDGE
service = Service()

options = webdriver.EdgeOptions()
options.add_argument("-inprivate")
driver = webdriver.Edge(service=service, options=options)
driver.maximize_window()

url = 'https://toolsworldb2b.meuspedidos.com.br/'

driver.get(url)

In [6]:
sheetCategorias = 'Links/CategoriasMTX.xlsx'
columns = ['Link']
categoriasLinks = kit_function.readExcel(sheetCategorias,columns)

In [ ]:
for link in categoriasLinks['Link']:
    print(link)

In [4]:
#Captura o título da categoria
cssSeletorTitle = '#catalogWrapper > div:nth-child(2) > div.Catalog__titleB2bContainer___JtAi4 > div > div.Catalog__titleB2bInner___G8ViQ'

titleCategoria = driver.find_element(By.CSS_SELECTOR , cssSeletorTitle).text
print(titleCategoria)

ABRAÇADEIRA
